In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<h1> 실버타운 입지조건 분석 프로젝트 - [위치정보 분석] </h1>

- 데이터 개수
  - 실버타운 : 269개
  - 편의시설 : 157,049개
  - 의료시설 : 77,444개

- 편의시설과 의료시설을 하나의 데이터 프레임으로 합치기
  - 시설분류코드 칼럼명 추가
    - 편의시설 : C
    - 의료시설 : M
  - '시설명' 칼럼명 통일하기
    - 편의시설 : 분류명 + 지점명
    - 의료시설 : 요양기관명
- 각 행별 고유 아이디 부여하기 : 실버타운아이디 / 시설아이디
- 위도 경도 값을 이용하여 거리 계산
  - 해버사인(Haversine) 공식 사용
  - 반경 1km 내에 (편의/의료)시설 데이터 필터링하기
  - 결과값으로 새로운 데이터 프레임 만들기
     - 칼럼구성 : 실버타운아이디, 실버타운명, 시설아이디, 시설명, 시설분류코드, 거리
-

In [ ]:
import pandas as pd

silver_town = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Github/Data_Analysis/SilverTown/전처리 이후 데이터/SilverTownProcessedData.csv')
convenient_facilities = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Github/Data_Analysis/SilverTown/전처리 이후 데이터/ConvenientFacilitiesProcessedData.csv')
medical_facilities = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Github/Data_Analysis/SilverTown/전처리 이후 데이터/MedicalFacilitiesProcessedData.csv')

In [ ]:
# 실버타운 아이디 부여

silver_town['실버타운아이디'] = range(1, len(silver_town) + 1)
silver_town

,시설명,시설유형,주소,경도,위도,실버타운아이디
0,청담실버타운5차,주택,서울특별시 강남구 청담동 13-41,127.045851,37.522260,1
1,신우실버타운,주택,서울특별시 강남구 삼성동 65-6,127.051894,37.517074,2
2,호세실버타운,노인복지,인천광역시 강화군 불은면 두운리 342-5,126.477926,37.675357,3
3,치악산실버타운,노인복지,강원특별자치도 횡성군 강림면 강림리 744-5,128.120994,37.357565,4
4,실버타운,노인복지,서울특별시 마포구 창전동 158-1,126.931016,37.545696,5
...,...,...,...,...,...,...
264,주안 에쉐르카운티,노인복지,인천광역시 미추홀구 주안동 1216-17,126.682657,37.445068,265
265,아산실버요양원,"요양원,요양센터",경상북도 경산시 진량읍 마곡리 165-1,128.846678,35.846351,266
266,민들레실버요양원,"요양원,요양센터",경상북도 영천시 화산면 대안리 204-2,128.827800,36.030962,267
267,백세만세실버케어,"요양원,요양센터",충청남도 보령시 청라면 장현리 570-1,126.673640,36.422286,268


In [ ]:
# '시설분류코드' 칼럼 추가
convenient_facilities['시설분류코드'] = 'C'
medical_facilities['시설분류코드'] = 'M'

# 편의시설 : 분류명+지점명→시설명 / 의료시설 : 요양기관명→시설명
convenient_facilities['시설명'] = convenient_facilities.apply(lambda row: f"{row['분류명']} {row['지점명']}", axis=1)
convenient_facilities = convenient_facilities.drop(columns=['분류명', '지점명'])
medical_facilities.rename(columns={'요양기관명':'시설명'}, inplace=True)

# 두 데이터프레임 합치기
facilities = pd.concat([convenient_facilities, medical_facilities], ignore_index=True)

# 시설 아이디 부여
facilities['시설아이디'] = range(1, len(facilities) + 1)

facilities

,분류아이디,분류코드,지번고유코드,경도,위도,주소,시설분류코드,시설명,종별코드명,우편번호,총의사수,시설아이디
0,KCDLCPO23N000070382,90702.0,4.146310e+18,127.108874,37.274116,경기도 용인시 기흥구 신갈동 신갈로84번길 29,C,GS25 신갈타운점,NaN,NaN,NaN,1
1,KCDLCPO23N000070383,90702.0,4.146310e+18,127.103098,37.274834,경기도 용인시 기흥구 신갈동 신정로41번길 1-1,C,GS25 신갈태광점,NaN,NaN,NaN,2
2,KCDLCPO23N000070384,90702.0,4.115010e+18,127.058630,37.726840,경기도 의정부시 신곡동 회룡로 223,C,GS25 신곡본점,NaN,NaN,NaN,3
3,KCDLCPO23N000070385,90702.0,4.115010e+18,127.055564,37.731400,경기도 의정부시 신곡동 발곡로 5,C,GS25 신곡으뜸점,NaN,NaN,NaN,4
4,KCDLCPO23N000070386,90702.0,4.115010e+18,127.063222,37.735586,경기도 의정부시 신곡동 시민로 256,C,GS25 신곡한일점,NaN,NaN,NaN,5
...,...,...,...,...,...,...,...,...,...,...,...,...
234488,NaN,NaN,NaN,NaN,NaN,"충청남도 논산시 중앙로491번길 12, (화지동)",M,힘센한의원,한의원,32950,1.0,234489
234489,NaN,NaN,NaN,126.968167,37.530005,"서울특별시 용산구 한강대로 109, (한강로2가, 용성비즈텔)",M,힘찬세상경희한의원,한의원,04376,1.0,234490
234490,NaN,NaN,NaN,127.434883,36.344682,"대전광역시 동구 계족로368번길 22, (성남동)",M,힘찬한의원,한의원,34590,1.0,234491
234491,NaN,NaN,NaN,128.064558,35.178636,"경상남도 진주시 진양호로 284, 2층 (신안동)",M,힘찬한의원,한의원,52697,1.0,234492


In [ ]:
subset_silver_town = silver_town.iloc[0:50] #1
# subset_silver_town = silver_town.iloc[50:100] #2
# subset_silver_town = silver_town.iloc[100:150] #3
# subset_silver_town = silver_town.iloc[150:200] #4
# subset_silver_town = silver_town.iloc[200:250] #5
# subset_silver_town = silver_town.iloc[250:269] #6
subset_silver_town

,시설명,시설유형,주소,경도,위도,실버타운아이디
0,청담실버타운5차,주택,서울특별시 강남구 청담동 13-41,127.045851,37.522260,1
1,신우실버타운,주택,서울특별시 강남구 삼성동 65-6,127.051894,37.517074,2
2,호세실버타운,노인복지,인천광역시 강화군 불은면 두운리 342-5,126.477926,37.675357,3
3,치악산실버타운,노인복지,강원특별자치도 횡성군 강림면 강림리 744-5,128.120994,37.357565,4
4,실버타운,노인복지,서울특별시 마포구 창전동 158-1,126.931016,37.545696,5
5,실버타운,주택,경상북도 경산시 자인면 북사리 259-4,128.820658,35.822469,6
6,더힐실버타운,노인복지,경기도 용인시 기흥구 동백동 381-12,127.156446,37.291500,7
7,워커힐실버타운,노인복지,서울특별시 광진구 구의동 16-3,127.097721,37.550708,8
8,한사랑실버타운,노인복지,광주광역시 북구 중흥동 348-14,126.909202,35.167957,9
9,서면 실버복지타운,노인복지,경상북도 청도군 화양읍 서상리 521,128.700587,35.653960,10


In [ ]:
import math

# 위도 경도 값으로 구하는 두 지점 간의 거리 계산
def haversine(lon1, lat1, lon2, lat2):

    # decimal degrees를 radians로 변환
    lon1, lat1, lon2, lat2 = map(math.radians, [lon1, lat1, lon2, lat2])

    # haversine 공식
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.asin(math.sqrt(a))
    r = 6371  # 지구의 평균 반지름 (단위: km)
    return c * r

In [ ]:
def main():

  distance_result = []

  for index, town in subset_silver_town.iterrows():
    print(index, ":", town['시설명'])
    for _, facility in facilities.iterrows():
      distance = haversine(pd.to_numeric(town['경도']), pd.to_numeric(town['위도']), pd.to_numeric(facility['경도']), pd.to_numeric(facility['위도']))
      result_data = {
            '실버타운아이디': town['실버타운아이디'],
            '실버타운명': town['시설명'],
            '시설아이디': facility['시설아이디'],
            '시설명': facility['시설명'],
            '시설분류코드' : facility['시설분류코드'],
            '거리': distance
          }
      distance_result.append(result_data)

  return distance_result

In [ ]:
if __name__ == '__main__':
  distance_df = main()
  distance_df = pd.DataFrame(distance_df)
  distance_df.to_csv('/content/drive/MyDrive/Colab Notebooks/Github/Data_Analysis/SilverTown/거리 계산 결과 데이터/DistanceData_subset1.csv', index=False)

0 : 청담실버타운5차
1 : 신우실버타운
2 : 호세실버타운
3 : 치악산실버타운
4 : 실버타운
5 : 실버타운
6 : 더힐실버타운
7 : 워커힐실버타운
8 : 한사랑실버타운
9 : 서면 실버복지타운
10 : 동해약천온천실버타운
11 : 늘푸른실버타운
12 : 삼성홈실버타운
13 : 레인보우빌리지 실버타운
14 : 천안실버타운
15 : 삼성실버타운주간보호센터
16 : 유당마을실버타운
17 : 청평태양실버타운
18 : 실버타운청심빌리지
19 : 에쉐르카운티 실버타운형 아파트
20 : 낙원실버타운&암자연치유
21 : 흰돌실버타운
22 : 해인실버타운
23 : 도원실버타운
24 : 황전실버타운
25 : 경주안심실버타운
26 : 케어닥케어홈 실버타운 송추포레스트점
27 : 효덕실버타운
28 : 초원실버타운
29 : 에덴실버타운
30 : 서민실버타운
31 : 홍광실버타운
32 : 성라실버타운
33 : 노엘실버타운
34 : 칠곡실버타운
35 : 성덕실버타운
36 : 미리내실버타운
37 : 겔러리온 실버타운
38 : 서포항실버타운
39 : 원불교하이원빌리지
40 : 대구 신세계주간보호센터
41 : 가나실버타운
42 : 왕지실버타운요양원.주간보호센터.
43 : 군포행복실버타운
44 : 근영실버타운
45 : 한국노인복지진흥재단포천실버타운
46 : 효사랑실버타운
47 : 글로리아실버타운
48 : 익산실버타운주간보호센터
49 : 한사랑실버타운


In [ ]:
# 전체 정렬 데이터 내보내기
import pandas as pd
import glob

distance_subset_files = glob.glob('/content/drive/MyDrive/Colab Notebooks/Github/Data_Analysis/SilverTown/거리 계산 결과 데이터/DistanceData_subset*.csv')
result = []

for file in distance_subset_files:
    df = pd.read_csv(file)
    df_sorted = df.sort_values(by=['실버타운아이디', '거리'])
    result.append(df_sorted)

# 데이터프레임 리스트를 하나의 데이터프레임으로 결합
combined_df = pd.concat(result)

combined_df.to_csv('/content/drive/MyDrive/Colab Notebooks/Github/Data_Analysis/SilverTown/거리 계산 결과 데이터/TotalSortedDistanceData.csv', index=False)

In [2]:
# 거리가 가까운 30개의 데이터씩만 내보내기
import pandas as pd
import glob

distance_subset_files = glob.glob('/content/drive/MyDrive/Colab Notebooks/Github/Data_Analysis/SilverTown/거리 계산 결과 데이터/DistanceData_subset*.csv')
result = []

for file in distance_subset_files:

    df = pd.read_csv(file)
    df_sorted = df.sort_values(by=['실버타운아이디', '거리'])

    top_30 = df_sorted.groupby('실버타운아이디').head(30)
    result.append(top_30)

# 데이터프레임 리스트를 하나의 데이터프레임으로 결합
combined_df = pd.concat(result)

combined_df.to_csv('/content/drive/MyDrive/Colab Notebooks/Github/Data_Analysis/SilverTown/거리 계산 결과 데이터/Top30DistanceData.csv', index=False)


<h1> 거리 데이터 분석 </h1>

In [5]:
top30_distance_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Github/Data_Analysis/SilverTown/거리 계산 결과 데이터/Top30DistanceData.csv')
top30_distance_df

,실버타운아이디,실버타운명,시설아이디,시설명,시설분류코드,거리
0,1,청담실버타운5차,54215,세탁소/빨래방 청담점,C,0.015099
1,1,청담실버타운5차,18595,세탁소/빨래방 nan,C,0.024679
2,1,청담실버타운5차,88364,이마트24 청담하나점,C,0.047058
3,1,청담실버타운5차,67309,세탁소/빨래방 nan,C,0.090885
4,1,청담실버타운5차,69320,GS25 청담타운점,C,0.101925
...,...,...,...,...,...,...
8065,269,푸른마을,217523,고창군흥덕면용반보건진료소,M,6.566293
8066,269,푸른마을,80854,CU 정읍스타점,C,6.579836
8067,269,푸른마을,5462,GS25 정읍아산병원점,C,6.656499
8068,269,푸른마을,52638,CU 정읍첨단점,C,6.675500
